# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "UID=WOODFIELD\\jmeza@wdsrx.com;"
                        "PWD=Clavezaza2022;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Oct 17 2022 8:30AM,248688,10,SOTAH0000811,"Nextsource Biotechnology, LLC",Released
1,Oct 14 2022 4:37PM,248689,18,INNOVA-8647393,Innova Softgel LLC,Released
2,Oct 14 2022 4:20PM,248687,16,SHL-8647389,"SHL Pharma, LLC",Released
3,Oct 14 2022 3:02PM,248683,10,7086603,"SHL Pharma, LLC",Released
4,Oct 14 2022 2:43PM,248679,12,8646865,HealthCaps LLC,Released
5,Oct 14 2022 2:29PM,248676,16,8642573,Sartorius Bioprocess Solutions,Completed
6,Oct 14 2022 2:29PM,248676,16,8645699,Sartorius Bioprocess Solutions,Executing
7,Oct 14 2022 2:29PM,248676,16,8645722,Sartorius Bioprocess Solutions,Released
8,Oct 14 2022 2:22PM,248673,16,SSF_C1W42,Sartorius Stedim Filters Inc.,Executing
9,Oct 14 2022 11:45AM,248642,16,ACG-2102491561,ACG North America LLC,Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
7,248679,Released,1
8,248683,Released,1
9,248687,Released,1
10,248688,Released,1
11,248689,Released,1


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Completed,Executing,Released
id,,,
248679,NaN,NaN,1.0
248683,NaN,NaN,1.0
248687,NaN,NaN,1.0
248688,NaN,NaN,1.0
248689,NaN,NaN,1.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
248621,0.0,1.0,5.0
248642,0.0,0.0,1.0
248673,0.0,1.0,0.0
248676,1.0,1.0,1.0
248679,0.0,0.0,1.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
248621,0,1,5
248642,0,0,1
248673,0,1,0
248676,1,1,1
248679,0,0,1


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,248621,0,1,5
1,248642,0,0,1
2,248673,0,1,0
3,248676,1,1,1
4,248679,0,0,1


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,248621,,1,5
1,248642,,,1
2,248673,,1,
3,248676,1,1,1
4,248679,,,1


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Oct 17 2022 8:30AM,248688,10,"Nextsource Biotechnology, LLC"
1,Oct 14 2022 4:37PM,248689,18,Innova Softgel LLC
2,Oct 14 2022 4:20PM,248687,16,"SHL Pharma, LLC"
3,Oct 14 2022 3:02PM,248683,10,"SHL Pharma, LLC"
4,Oct 14 2022 2:43PM,248679,12,HealthCaps LLC
5,Oct 14 2022 2:29PM,248676,16,Sartorius Bioprocess Solutions
8,Oct 14 2022 2:22PM,248673,16,Sartorius Stedim Filters Inc.
9,Oct 14 2022 11:45AM,248642,16,ACG North America LLC
10,Oct 14 2022 10:08AM,248621,10,Eywa Pharma Inc.


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Completed,Executing,Released
0,Oct 17 2022 8:30AM,248688,10,"Nextsource Biotechnology, LLC",,,1
1,Oct 14 2022 4:37PM,248689,18,Innova Softgel LLC,,,1
2,Oct 14 2022 4:20PM,248687,16,"SHL Pharma, LLC",,,1
3,Oct 14 2022 3:02PM,248683,10,"SHL Pharma, LLC",,,1
4,Oct 14 2022 2:43PM,248679,12,HealthCaps LLC,,,1
5,Oct 14 2022 2:29PM,248676,16,Sartorius Bioprocess Solutions,1,1,1
6,Oct 14 2022 2:22PM,248673,16,Sartorius Stedim Filters Inc.,,1,
7,Oct 14 2022 11:45AM,248642,16,ACG North America LLC,,,1
8,Oct 14 2022 10:08AM,248621,10,Eywa Pharma Inc.,,1,5


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Oct 17 2022 8:30AM,248688,10,"Nextsource Biotechnology, LLC",1,,
1,Oct 14 2022 4:37PM,248689,18,Innova Softgel LLC,1,,
2,Oct 14 2022 4:20PM,248687,16,"SHL Pharma, LLC",1,,
3,Oct 14 2022 3:02PM,248683,10,"SHL Pharma, LLC",1,,
4,Oct 14 2022 2:43PM,248679,12,HealthCaps LLC,1,,
5,Oct 14 2022 2:29PM,248676,16,Sartorius Bioprocess Solutions,1,1,1
6,Oct 14 2022 2:22PM,248673,16,Sartorius Stedim Filters Inc.,,1,
7,Oct 14 2022 11:45AM,248642,16,ACG North America LLC,1,,
8,Oct 14 2022 10:08AM,248621,10,Eywa Pharma Inc.,5,1,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Oct 17 2022 8:30AM,248688,10,"Nextsource Biotechnology, LLC",1,,
1,Oct 14 2022 4:37PM,248689,18,Innova Softgel LLC,1,,
2,Oct 14 2022 4:20PM,248687,16,"SHL Pharma, LLC",1,,
3,Oct 14 2022 3:02PM,248683,10,"SHL Pharma, LLC",1,,
4,Oct 14 2022 2:43PM,248679,12,HealthCaps LLC,1,,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Oct 17 2022 8:30AM,248688,10,"Nextsource Biotechnology, LLC",1.0,NaN,NaN
1,Oct 14 2022 4:37PM,248689,18,Innova Softgel LLC,1.0,NaN,NaN
2,Oct 14 2022 4:20PM,248687,16,"SHL Pharma, LLC",1.0,NaN,NaN
3,Oct 14 2022 3:02PM,248683,10,"SHL Pharma, LLC",1.0,NaN,NaN
4,Oct 14 2022 2:43PM,248679,12,HealthCaps LLC,1.0,NaN,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Oct 17 2022 8:30AM,248688,10,"Nextsource Biotechnology, LLC",1.0,0.0,0.0
1,Oct 14 2022 4:37PM,248689,18,Innova Softgel LLC,1.0,0.0,0.0
2,Oct 14 2022 4:20PM,248687,16,"SHL Pharma, LLC",1.0,0.0,0.0
3,Oct 14 2022 3:02PM,248683,10,"SHL Pharma, LLC",1.0,0.0,0.0
4,Oct 14 2022 2:43PM,248679,12,HealthCaps LLC,1.0,0.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing,Completed
Warehouse,,,,
10,745992,7.0,1.0,0.0
12,248679,1.0,0.0,0.0
16,994678,3.0,2.0,1.0
18,248689,1.0,0.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing,Completed
0,10,745992,7.0,1.0,0.0
1,12,248679,1.0,0.0,0.0
2,16,994678,3.0,2.0,1.0
3,18,248689,1.0,0.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing,Completed
0,10,7.0,1.0,0.0
1,12,1.0,0.0,0.0
2,16,3.0,2.0,1.0
3,18,1.0,0.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,7.0
1,12,Released,1.0
2,16,Released,3.0
3,18,Released,1.0
4,10,Executing,1.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,12,16,18
Status,,,,
Completed,0.0,0.0,1.0,0.0
Executing,1.0,0.0,2.0,0.0
Released,7.0,1.0,3.0,1.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,12,16,18
0,Completed,0.0,0.0,1.0,0.0
1,Executing,1.0,0.0,2.0,0.0
2,Released,7.0,1.0,3.0,1.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,12,16,18
0,Completed,0.0,0.0,1.0,0.0
1,Executing,1.0,0.0,2.0,0.0
2,Released,7.0,1.0,3.0,1.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()